In [1]:
import pandas as pd
import numpy as np
import re
from random import random
import emoji
from tqdm import notebook
def tqdm(x, **kargs):
    return notebook.tqdm(x, leave=False, **kargs)
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import torch
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from os import listdir
import csv
import json

In [2]:
from simpletransformers.classification import MultiLabelClassificationModel, ClassificationModel
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder

In [3]:
from simpletransformers.language_modeling import LanguageModelingModel

## Read input data

In [4]:
# train_root='source/preprocess_train.json'
train_root='evaluation/source/preprocess_train.json'
data = [json.loads(line) for line in open(train_root, 'r')]   
df_new = pd.DataFrame(data)
df_new.head()

,idx,text,reply,label
0,8981,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ GenFlynn @ BarbaraRedgate...,fake
1,8981,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ GenFlynn @ BarbaraRedgate...,fake
2,8981,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ GenFlynn @ BarbaraRedgate...,fake
3,8981,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ Dean - Moriarty36 @ Gen...,fake
4,8981,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ kldreams61 @ GenFlynn @ ...,fake


In [5]:
labelencoder = LabelEncoder()
categories_type = df_new['label'].copy()
categories_type= labelencoder.fit_transform(categories_type)
categories_type

array([0, 0, 0, ..., 0, 0, 0])

In [6]:
len(np.where(categories_type==0)[0])/len(categories_type)

0.8113054159422268

In [11]:
# df_dev_ori = pd.read_json('./source/dev_unlabeled.json', lines=True)
# df_dev = pd.read_json('./preprocessed/preprocess_dev.json', lines=True)
# df_dev_result = df_dev.copy()[['text', 'reply']]
# df_dev_result.head()


# dev_root='source/preprocess_dev.json'
dev_root='evaluation/source/preprocess_dev.json'
dev_data = [json.loads(line) for line in open(dev_root, 'r')]   
df_dev = pd.DataFrame(dev_data)
df_dev_result = df_dev.copy()[['text', 'reply']]
df_dev_result.head()

,text,reply
0,If there was any moment that I needed love and...,"Hey , Court ! ! If you need anything , you..."
1,Hi Guys up for a chat . . . . . if yes s...,Yeahh . . . . . We are Waiting\n # AskSid
2,Did I get myself physically stuck in a filing ...,
3,Day one on medication and I no longer have dep...,
4,"Drop your cash app , use hashtag # BailoutHu...",$ tyratomaro # BailoutHumans


In [13]:
# df_test_ori = pd.read_json('./source/test_unlabeled.json', lines=True)
# df_test = pd.read_json('./preprocessed/preprocess_test.json', lines=True)
# df_test_result = df_test.copy()[['text', 'reply']]
# df_test_result.head()
# test_root='source/preprocess_test.json'
test_root='evaluation/source/preprocess_test.json'
test_data = [json.loads(line) for line in open(test_root, 'r')]   
df_test = pd.DataFrame(test_data)
df_test_result = df_test.copy()[['text', 'reply']]
df_test_result.head()

,text,reply
0,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Trump Warns Iranian P...
1,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Why Did ' Russian ...
2,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN This is nothing but t...
3,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Good to see you @ Re...
4,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Israeli Jets Strike S...


### Use multi-hot encoding and change column name

In [14]:
df_new.head()

,idx,text,reply,label
0,8981,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ GenFlynn @ BarbaraRedgate...,fake
1,8981,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ GenFlynn @ BarbaraRedgate...,fake
2,8981,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ GenFlynn @ BarbaraRedgate...,fake
3,8981,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ Dean - Moriarty36 @ Gen...,fake
4,8981,How incredibly despicable these traitors are ...,@ JosephJFlynn1 @ kldreams61 @ GenFlynn @ ...,fake


In [10]:
# df_new['labels'] = multi_hot_list
# df_new = df_new[['text', 'reply', 'labels']]
# df_new.columns = ['text_a', 'text_b', 'labels']
# df_dev_result.columns = ['text_a', 'text_b']
# df_test_result.columns = ['text_a', 'text_b']
df_new['labels'] = categories_type
df_new = df_new[['text', 'reply', 'labels']]

df_new.columns = ['text_a', 'text_b', 'labels']
# df_dev_result.columns = ['text_a', 'text_b']
# df_test_result.columns = ['text_a', 'text_b']

KeyError: "['text', 'reply'] not in index"

# 10 roberta

In [19]:
# Output data to train_file for training LM
# import wandb
# run = wandb.init()


df_forLM = df_new.copy()
df_forLM['text'] = ['<s> '] + df_new['text_a'] + [' </s></s> '] + df_new['text_b'] + [' <s>']
df_forLM['text'].to_csv(r'evaluation/LM/training.txt', header=None, index=None, sep=' ')
for i in range(1, 2):
    train_lm_args = {
        "output_dir": "evaluation/round2_eval_model/round2_LM/outputs_roberta_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "evaluation/LM/outputs/best_model/",
        "block_size": 512,
        "overwrite_output_dir": True,
        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 113,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,
    }
    
    model = LanguageModelingModel("roberta", "roberta-base", args=train_lm_args)
    model.train_model('LM/training.txt')
    
#     train_args = {
#         "output_dir": "round1_blend_model/round1_MLR/outputs_roberta_LM_training{}/".format(i),
#         "cache_dir": "cache/",
#         "best_model_dir": "model_results_gpu/outputs/best_model/",
#         "block_size": 512,
#         "fp16": False,
#         "fp16_opt_level": "O1",
#         "max_seq_length": 113,
#         "train_batch_size": 16,
#         "eval_batch_size": 16,
#         "gradient_accumulation_steps": 1,
#         "num_train_epochs": 4,
#         "weight_decay": 0,
#         "learning_rate": 4e-5,
#         "adam_epsilon": 1e-8,
#         "warmup_ratio": 0.06,
#         "warmup_steps": 0,
#         "max_grad_norm": 1.0,
#         "do_lower_case": False,

#         "logging_steps": 50,
#         "evaluate_during_training": False,
#         "evaluate_during_training_steps": 2000,
#         "evaluate_during_training_verbose": False,
#         "use_cached_eval_features": False,
#         "save_eval_checkpoints": False,
#         "save_steps": 2000,
#         "no_cache": True,
#         "save_model_every_epoch": True,
#         "tensorboard_dir": None,

#         "overwrite_output_dir": True,
#         "reprocess_input_data": True,

#         "n_gpu": 1,
#         "silent": False,
#         "use_multiprocessing": False,

#         "wandb_project": None,
#         "wandb_kwargs": {},

#         "use_early_stopping": True,
#         "early_stopping_patience": 3,
#         "early_stopping_delta": 0,
#         "early_stopping_metric": "eval_loss",
#         "early_stopping_metric_minimize": True,

#         "manual_seed": None,
#         "encoding": None,
#         "config": {}
#     }
#     model = ClassificationModel('roberta', 'round1_blend_model/round1_LM/outputs_roberta_LM_training{}/'.format(i), num_labels=2,weight=[0.2,0.8], args=train_args)
#     model.train_model(df_new)

KeyboardInterrupt: 

In [24]:
# from sklearn.utils import shuffle
# df_new = shuffle(df_new)

# split_idx=len(df_new)//10*8
# df_val=df_new.iloc[split_idx:]
# df_new=df_new.iloc[split_idx:]
# # df_new.head()

In [40]:
df_new.head()

,text_a,text_b,labels
161713,Hi . Could all commentators please get it rig...,@ AmandaFBelfast Generous of ye to call them ...,0
123518,"New York Times asks "" How Can I Cure My Whi...",@ TheRebelTV @ MartinaMarkota Such a shame t...,0
59222,Cats are natural carpenters because their tong...,@ jlsigman4 Haha I am not sure who Burt is bu...,0
157466,Do you care about me ? \n . \n . \n . \nWh...,"Yes , I do 👉🏼❤️👈🏼",1
77472,I have just spoken to an authority person from...,""" Azerberjan have try to get Shushi today b...",0


In [11]:
# df_new
df_real=df_new[df_new["labels"]==1]
df_fake=df_new[df_new["labels"]==0]

In [13]:
len(df_real)

31799

In [14]:
len(df_fake)

136722

In [22]:
# train_lm_args = {
#         "output_dir": "round1_blend_model/round1_LM/outputs_roberta_LM_training{}/".format(2),
#         "cache_dir": "cache/",
#         "best_model_dir": "LM/outputs/best_model/",
#         "block_size": 512,
#         "overwrite_output_dir": True,
#         "fp16": False,
#         "fp16_opt_level": "O1",
#         "max_seq_length": 113,
#         "train_batch_size": 16,
#         "eval_batch_size": 16,
#         "gradient_accumulation_steps": 1,
#         "num_train_epochs": 4,
#         "weight_decay": 0,
#         "learning_rate": 4e-5,
#         "adam_epsilon": 1e-8,
#         "warmup_ratio": 0.06,
#         "warmup_steps": 0,
#         "max_grad_norm": 1.0,
#         "do_lower_case": False,
#     }
    
# model = LanguageModelingModel("roberta", "roberta-base", args=train_lm_args)
# model.train_model('LM/training.txt')

In [26]:
from sklearn.utils import shuffle
for i in range(1,5):
    
#     df_fake=df_new[df_new['labels']==0]
#     df_real=df_new[df_new['labels']==1]
#     real_number=len(df_new[df_new['labels']==1])
#     df_fake=df_fake.sample(n=real_number)
#     df_sample=pd.concat([df_fake, df_real])
    
#     df_fake=df_fake.sample(n=real_number)
#     df_eval=pd.concat([df_fake, df_real])


    df_newnew=df_fake.sample(n=len(df_real))
    df_newnew=pd.concat([df_newnew,df_real])
    df_newnew = shuffle(df_newnew)


    train_args = {
        "output_dir": "evaluation/round2_eval_model/round2_MLR/outputs_roberta_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "model_results_gpu/outputs/best_model/",
        "block_size": 512,
        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 113,

        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "logging_steps": 50,

        "evaluate_during_training": False,
        "evaluate_during_training_steps": 2000,
        "evaluate_during_training_verbose": True,

        "use_cached_eval_features": False,
        "save_eval_checkpoints": False,
        "save_steps": 2000,
        "no_cache": True,
        "save_model_every_epoch": True,
        "tensorboard_dir": None,

        "overwrite_output_dir": True,
        "reprocess_input_data": True,

        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": False,

        "wandb_project": None,
        "wandb_kwargs": {},

        "use_early_stopping": True,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0,
        "early_stopping_metric": "eval_loss",
        "early_stopping_metric_minimize": True,

        "manual_seed": None,
        "encoding": None,
        "config": {}
    }
    model = ClassificationModel('roberta', "evaluation/round2_eval_model/round2_LM/outputs_roberta_LM_training{}/".format(1), num_labels=2,args=train_args)
    model.train_model(df_newnew)

Some weights of the model checkpoint at evaluation/round2_eval_model/round2_LM/outputs_roberta_LM_training1/ were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at evaluation/round2_eval_model/round2_LM/outputs_roberta_LM_

Some weights of the model checkpoint at evaluation/round2_eval_model/round2_LM/outputs_roberta_LM_training1/ were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at evaluation/round2_eval_model/round2_LM/outputs_roberta_LM_

Some weights of the model checkpoint at evaluation/round2_eval_model/round2_LM/outputs_roberta_LM_training1/ were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at evaluation/round2_eval_model/round2_LM/outputs_roberta_LM_

Some weights of the model checkpoint at evaluation/round2_eval_model/round2_LM/outputs_roberta_LM_training1/ were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at evaluation/round2_eval_model/round2_LM/outputs_roberta_LM_

In [88]:
result, model_outputs, wrong_predictions = model.eval_model(df_val)

In [68]:
result

{'mcc': 0.9929666580784563,
 'tp': 6371,
 'tn': 27261,
 'fp': 33,
 'fn': 40,
 'auroc': 0.998677142679851,
 'auprc': 0.9977671578395382,
 'eval_loss': 0.01605951650630405}

In [21]:
# model = ClassificationModel('roberta', 'round1_blend_model/round1_MLR/outputs_roberta_LM_training1')

In [78]:
df_test_result.head()

,text_a,text_b
0,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Trump Warns Iranian P...
1,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Why Did ' Russian ...
2,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN This is nothing but t...
3,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Good to see you @ Re...
4,If a passerby had not posted # HillaryClinton...,@ RealJamesWoods @ CNN Israeli Jets Strike S...


In [35]:
len(dev_data)

88664

In [29]:
raw_prediction_list=[]

with open('evaluation/pubilc_data/eval.json',encoding="utf-8") as g:
    dev_data = json.load(g)

df_dev = pd.DataFrame(dev_data)

to_predict = []
for i in range(len(df_test_result)):
    text_reply = '<s> ' + df_test_result['text_a'][i] + ' </s></s> ' + df_test_result['text_b'][i] + ' <s>'
    to_predict.append(text_reply)

print(len(to_predict))
print(len(dev_data))

path="evaluation/round2_eval_model/round2_MLR/outputs_roberta_LM_training{}/"

model = ClassificationModel('roberta', path.format(1))
predictions_roberta_base, raw_outputs_roberta_base = model.predict(to_predict)
raw_prediction_list.append(raw_outputs_roberta_base)

88664
88664


In [38]:
# raw_prediction_list.append(raw_outputs_roberta_base)

In [30]:
model = ClassificationModel('roberta', path.format(2))
predictions_roberta_base, raw_outputs_roberta_base = model.predict(to_predict)
raw_prediction_list.append(raw_outputs_roberta_base)

In [31]:
model = ClassificationModel('roberta', path.format(3))
predictions_roberta_base, raw_outputs_roberta_base = model.predict(to_predict)
raw_prediction_list.append(raw_outputs_roberta_base)

In [32]:
model = ClassificationModel('roberta', path.format(4))
predictions_roberta_base, raw_outputs_roberta_base = model.predict(to_predict)
raw_prediction_list.append(raw_outputs_roberta_base)

In [33]:
raw_prediction_list=np.array(raw_prediction_list)
average=np.mean(raw_prediction_list,axis=0)
temp_list=[]
for i in range(average.shape[0]):
    if average[i,0]>average[i,1]:
        temp_list.append(0)
    else:
        temp_list.append(1)

In [34]:
print(len(temp_list))

88664


In [35]:
sum(temp_list)/len(temp_list)

0.3152237661283046

In [36]:
idx_list=[]
context_idx_list=[]
lable_list=[]
for idx, i in enumerate(temp_list):
    idx_list.append(df_dev["idx"].iloc[idx])
    context_idx_list.append(df_dev["context_idx"].iloc[idx])
    if i==0:
        lable_list.append("fake")
    else:
        lable_list.append("real")

d = {'idx': idx_list, 'context_idx': context_idx_list ,"label":lable_list}
df = pd.DataFrame(data=d)
df.to_csv("evaluation/eval.csv",index=False)

In [44]:
len(df)

88664

# 5 cased

In [ ]:
df_forLM = df_new.copy()
df_forLM['text'] = ['[CLS] '] + df_new['text_a'] + [' [SEP] '] + df_new['text_b'] + [' [SEP]']
df_forLM['text'].to_csv(r'/content/drive/Shareddrives/NLP/LM/training.txt', header=None, index=None, sep=' ')
for i in range(1, 6):
    train_lm_args = {
        "output_dir": "round1_blend_model/round1_LM/outputs_bert_cased_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 113,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,
    }
    model = LanguageModelingModel("bert", "bert-base-cased", args=train_lm_args)
    model.train_model('/content/drive/Shareddrives/NLP/LM/training.txt')
    train_args = {
        "output_dir": "/content/drive/Shareddrives/NLP/round1_blend_model/round1_MLR/outputs_bert_cased_LM_training{}/".format(i),
        "cache_dir": "/content/drive/Shareddrives/NLP/cache/",
        "best_model_dir": "/content/drive/Shareddrives/NLP/model_results_gpu/outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 113,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "logging_steps": 50,
        "evaluate_during_training": False,
        "evaluate_during_training_steps": 2000,
        "evaluate_during_training_verbose": False,
        "use_cached_eval_features": False,
        "save_eval_checkpoints": False,
        "save_steps": 2000,
        "no_cache": True,
        "save_model_every_epoch": True,
        "tensorboard_dir": None,

        "overwrite_output_dir": False,
        "reprocess_input_data": True,

        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": False,

        "wandb_project": None,
        "wandb_kwargs": {},

        "use_early_stopping": True,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0,
        "early_stopping_metric": "eval_loss",
        "early_stopping_metric_minimize": True,

        "manual_seed": None,
        "encoding": None,
        "config": {}
    }
    model = MultiLabelClassificationModel('bert', '/content/drive/Shareddrives/NLP/round1_blend_model/round1_LM/outputs_bert_cased_LM_training{}/'.format(i), num_labels=2, args=train_args)
    model.train_model(df_new)

FileNotFoundError: ignored

# 5 uncased

In [ ]:
df_forLM = df_new.copy()
df_forLM['text'] = ['[CLS] '] + df_new['text_a'] + [' [SEP] '] + df_new['text_b'] + [' [SEP]']
df_forLM['text'].to_csv(r'/content/drive/Shareddrives/NLP/LM/training.txt', header=None, index=None, sep=' ')
for i in range(1, 6):
    train_lm_args = {
        "output_dir": "/content/drive/Shareddrives/NLP/round1_blend_model/round1_LM/outputs_bert_uncased_LM_training{}/".format(i),
        "cache_dir": "/content/drive/Shareddrives/NLP/cache/",
        "best_model_dir": "/content/drive/Shareddrives/NLP/LM/outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 113,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,
    }
    model = LanguageModelingModel("bert", "bert-base-uncased", args=train_lm_args)
    model.train_model('/content/drive/Shareddrives/NLP/LM/training.txt')
    train_args = {
        "output_dir": "/content/drive/Shareddrives/NLP/round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training{}/".format(i),
        "cache_dir": "/content/drive/Shareddrives/NLP/cache/",
        "best_model_dir": "/content/drive/Shareddrives/NLP/model_results_gpu/outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 113,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "logging_steps": 50,
        "evaluate_during_training": False,
        "evaluate_during_training_steps": 2000,
        "evaluate_during_training_verbose": False,
        "use_cached_eval_features": False,
        "save_eval_checkpoints": False,
        "save_steps": 2000,
        "no_cache": True,
        "save_model_every_epoch": True,
        "tensorboard_dir": None,

        "overwrite_output_dir": True,
        "reprocess_input_data": True,

        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": False,

        "wandb_project": None,
        "wandb_kwargs": {},

        "use_early_stopping": True,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0,
        "early_stopping_metric": "eval_loss",
        "early_stopping_metric_minimize": True,

        "manual_seed": None,
        "encoding": None,
        "config": {}
    }
    model = MultiLabelClassificationModel('bert', '/content/drive/Shareddrives/NLP/round1_blend_model/round1_LM/outputs_bert_uncased_LM_training{}/'.format(i), num_labels=2, args=train_args)
    model.train_model(df_new)

FileNotFoundError: ignored

## Predict results [pair sentance](https://simpletransformers.ai/docs/classification-data-formats/)

### test

In [ ]:
to_predict = []
for i in range(len(df_test_result)):
    text_reply = '<s> ' + df_test_result['text_a'][i] + ' </s></s> ' + df_test_result['text_b'][i] + ' <s>'
    to_predict.append(text_reply)

NameError: ignored

In [ ]:
for model_idx in range(1, 11):
    model_roberta_base = MultiLabelClassificationModel('roberta', 'round1_blend_model/round1_MLR/outputs_roberta_LM_training' + str(model_idx) + '/')
    predictions_roberta_base, raw_outputs_roberta_base = model_roberta_base.predict(to_predict)
    with open("./final_results/test/roberta_" + str(model_idx) + ".csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(raw_outputs_roberta_base)

In [ ]:
to_predict = []
for i in range(len(df_test_result)):
    text_reply = '[CLS] ' + df_test_result['text_a'][i] + ' [SEP] ' + df_test_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

In [ ]:
for bert_type in ['uncased', 'cased']:
    for model_idx in range(1, 6):
        model_bert_base = MultiLabelClassificationModel('bert', 'round1_blend_model/round1_MLR/outputs_bert_' + bert_type + '_LM_training' + str(model_idx) + '/')
        predictions_bert_base, raw_outputs_bert_base = model_bert_base.predict(to_predict)
        with open("./final_results/test/bert_" + bert_type + "_" + str(model_idx) + ".csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(raw_outputs_bert_base)

In [ ]:
files = listdir('./final_results/test/')
roberta_prob = np.zeros([len(df_test_result), len(categories_mapping)])
bert_uncase_prob = np.zeros([len(df_test_result), len(categories_mapping)])
bert_case_prob = np.zeros([len(df_test_result), len(categories_mapping)])
for f in files:
    if f.find('.') != 0:
        model_prob = pd.read_csv('./final_results/test/' + f, header=None)
        probability_list = model_prob.values.tolist()
        if f.find('roberta') >= 0:
            roberta_prob = np.add(roberta_prob, probability_list)
        if f.find('_cased_') >= 0:
            bert_case_prob = np.add(bert_case_prob, probability_list)
        if f.find('uncased') >= 0:
            bert_uncase_prob = np.add(bert_uncase_prob, probability_list)

In [ ]:
roberta_count = 10
bert_count = 5
roberta_prob /= roberta_count
bert_case_prob /= bert_count
bert_uncase_prob /= bert_count

In [ ]:
P = 3.5
total_predict = []
for eid, row_roberta_base in enumerate(roberta_prob):
    row = (row_roberta_base ** P) * 3.0 + \
            (bert_case_prob[eid] ** P) * 0.8 + (bert_uncase_prob[eid] ** P) * 0.8 
    predict_class = []
    correct = 0
    sort_index = sorted(range(len(row)), key=lambda k: row[k], reverse=True)
    for key, value in categories_mapping.items():
        if value in sort_index[0:6]:
            predict_class.append(key)
            
    total_predict.append(predict_class)

In [ ]:
df_test_ori['categories'] = total_predict
df_test_ori.head()

In [ ]:
df_test_ori.to_json('./results/eval.json', orient='records', lines=True)

# dev

In [ ]:
to_predict = []
for i in range(len(df_dev_result)):
    text_reply = '<s> ' + df_dev_result['text_a'][i] + ' </s></s> ' + df_dev_result['text_b'][i] + ' <s>'
    to_predict.append(text_reply)

In [ ]:
for model_idx in range(1, 11):
    model_roberta_base = MultiLabelClassificationModel('roberta', 'round1_blend_model/round1_MLR/outputs_roberta_LM_training' + str(model_idx) + '/')
    predictions_roberta_base, raw_outputs_roberta_base = model_roberta_base.predict(to_predict)
    with open("./final_results/dev/roberta_" + str(model_idx) + ".csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(raw_outputs_roberta_base)

I0625 20:34:01.536793 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training1/config.json
I0625 20:34:01.537540 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:34:17.969518 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training2/config.json
I0625 20:34:17.970340 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:34:20.560892 140553350014784 tokenization_utils.py:929] Model name 'round1_blend_model/round1_MLR/outputs_roberta_LM_training2/' not found in model shortcut name list (roberta-base, roberta-large, roberta-large-mnli, distilroberta-base, roberta-base-openai-detector, roberta-large-openai-detector). Assuming 'round1_blend_model/round1_MLR/outputs_roberta_LM_training2/' is a path, a model identifier, or url to a directory containing tokenizer files.
I0625 20:34:20.561494 140553350014784 tokenization_utils.py:958] Didn't find file round1_blend_model/round1_MLR/outputs_roberta_LM_training2/added_tokens.json. We won't load it.
I0625 20:34:20.561860 140553350014784 tokenization_utils.py:1013] loading file round1_blend_model/round1_MLR/outputs_roberta_LM_training2/vocab.json
I0625 20:34:20.562178 140553350014784 tokenization_utils.py:1013] loading file round1_blend_model/round1_MLR/outputs_roberta_LM_training2/merges.txt
I0625 20:34:20.562509 140553350014784 tokenization_utils.py:1013

I0625 20:34:34.621426 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training3/config.json
I0625 20:34:34.622107 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:34:51.088480 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training4/config.json
I0625 20:34:51.089193 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

...

I0625 20:35:40.752967 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training7/config.json
I0625 20:35:40.753614 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:35:57.272168 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training8/config.json
I0625 20:35:57.272832 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:36:13.788202 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training9/config.json
I0625 20:36:13.788918 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0625 20:36:30.441080 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_roberta_LM_training10/config.json


I0625 20:36:30.441716 140553350014784 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27",
    "28": "LABEL_28",
    "29": "LABEL_29",
    "30": "LABEL_

In [ ]:
to_predict = []
for i in range(len(df_dev_result)):
    text_reply = '[CLS] ' + df_dev_result['text_a'][i] + ' [SEP] ' + df_dev_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

In [ ]:
for bert_type in ['uncased', 'cased']:
    for model_idx in range(1, 6):
        model_bert_base = MultiLabelClassificationModel('bert', 'round1_blend_model/round1_MLR/outputs_bert_' + bert_type + '_LM_training' + str(model_idx) + '/')
        predictions_bert_base, raw_outputs_bert_base = model_bert_base.predict(to_predict)
        with open("./final_results/dev/bert_" + bert_type + "_" + str(model_idx) + ".csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(raw_outputs_bert_base)

I0625 20:41:35.630744 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training1/config.json
I0625 20:41:35.631663 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:41:51.503073 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training2/config.json
I0625 20:41:51.503672 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:42:07.455550 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training3/config.json
I0625 20:42:07.456245 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:42:23.333124 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training4/config.json
I0625 20:42:23.333827 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:42:39.393156 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_uncased_LM_training5/config.json
I0625 20:42:39.393832 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0625 20:42:55.296054 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training1/config.json
I0625 20:42:55.296774 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0625 20:43:11.231738 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training2/config.json
I0625 20:43:11.232401 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0625 20:43:27.290642 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training3/config.json
I0625 20:43:27.291420 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0625 20:43:43.246163 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training4/config.json
I0625 20:43:43.246868 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0625 20:43:59.186888 140553350014784 configuration_utils.py:283] loading configuration file round1_blend_model/round1_MLR/outputs_bert_cased_LM_training5/config.json
I0625 20:43:59.187621 140553350014784 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

In [ ]:
files = listdir('./final_results/dev/')
roberta_prob = np.zeros([len(df_dev_result), len(categories_mapping)])
bert_uncase_prob = np.zeros([len(df_dev_result), len(categories_mapping)])
bert_case_prob = np.zeros([len(df_dev_result), len(categories_mapping)])
for f in files:
    if f.find('.') != 0:
        model_prob = pd.read_csv('./final_results/dev/' + f, header=None)
        probability_list = model_prob.values.tolist()
        if f.find('roberta') >= 0:
            roberta_prob = np.add(roberta_prob, probability_list)
        if f.find('_cased_') >= 0:
            bert_case_prob = np.add(bert_case_prob, probability_list)
        if f.find('uncased') >= 0:
            bert_uncase_prob = np.add(bert_uncase_prob, probability_list)

In [ ]:
roberta_count = 10
bert_count = 5
roberta_prob /= roberta_count
bert_case_prob /= bert_count
bert_uncase_prob /= bert_count

In [ ]:
P = 3.5
total_predict = []
for eid, row_roberta_base in enumerate(roberta_prob):
    row = (row_roberta_base ** P) * 3.0 + \
            (bert_case_prob[eid] ** P) * 0.8 + (bert_uncase_prob[eid] ** P) * 0.8 
    predict_class = []
    correct = 0
    sort_index = sorted(range(len(row)), key=lambda k: row[k], reverse=True)
    for key, value in categories_mapping.items():
        if value in sort_index[0:6]:
            predict_class.append(key)
            
    total_predict.append(predict_class)

In [ ]:
df_dev_ori['categories'] = total_predict
df_dev_ori.head()
df_dev_ori.to_json('./results/dev.json', orient='records', lines=True)